In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv("fraudTrain.csv")
df = pd.DataFrame(train_data)
print(df)

         Unnamed: 0 trans_date_trans_time               cc_num  \
0                 0   2019-01-01 00:00:18     2703186189652095   
1                 1   2019-01-01 00:00:44         630423337322   
2                 2   2019-01-01 00:00:51       38859492057661   
3                 3   2019-01-01 00:01:16     3534093764340240   
4                 4   2019-01-01 00:03:06      375534208663984   
...             ...                   ...                  ...   
1296670     1296670   2020-06-21 12:12:08       30263540414123   
1296671     1296671   2020-06-21 12:12:19     6011149206456997   
1296672     1296672   2020-06-21 12:12:32     3514865930894695   
1296673     1296673   2020-06-21 12:13:36     2720012583106919   
1296674     1296674   2020-06-21 12:13:37  4292902571056973207   

                                    merchant       category     amt  \
0                 fraud_Rippin, Kub and Mann       misc_net    4.97   
1            fraud_Heller, Gutmann and Zieme    grocery_pos  107.

In [3]:
from copy import deepcopy
actual = deepcopy(df)

In [4]:

print(df.head(15))

    Unnamed: 0 trans_date_trans_time            cc_num  \
0            0   2019-01-01 00:00:18  2703186189652095   
1            1   2019-01-01 00:00:44      630423337322   
2            2   2019-01-01 00:00:51    38859492057661   
3            3   2019-01-01 00:01:16  3534093764340240   
4            4   2019-01-01 00:03:06   375534208663984   
5            5   2019-01-01 00:04:08  4767265376804500   
6            6   2019-01-01 00:04:42    30074693890476   
7            7   2019-01-01 00:05:08  6011360759745864   
8            8   2019-01-01 00:05:18  4922710831011201   
9            9   2019-01-01 00:06:01  2720830304681674   
10          10   2019-01-01 00:06:23     4642894980163   
11          11   2019-01-01 00:06:53   377234009633447   
12          12   2019-01-01 00:06:56   180042946491150   
13          13   2019-01-01 00:07:27  5559857416065248   
14          14   2019-01-01 00:09:03  3514865930894695   

                              merchant       category     amt        fi

In [5]:
numeric_df = df.select_dtypes(include=['float64', 'int64'])
correlation = numeric_df.corr(method='pearson')
correlation

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
Unnamed: 0,1.000000,0.000386,-0.000251,0.000709,0.000602,-0.000676,-0.001678,0.998971,0.000541,-0.000671,-0.004767
cc_num,0.000386,1.000000,0.001769,0.041459,-0.059271,-0.048278,-0.008991,0.000354,-0.058942,-0.048252,-0.000981
amt,-0.000251,0.001769,1.000000,0.001843,-0.001926,-0.000187,0.005818,-0.000293,-0.001873,-0.000151,0.219404
zip,0.000709,0.041459,0.001843,1.000000,-0.114290,-0.909732,0.078467,0.000670,-0.113561,-0.908924,-0.002162
lat,0.000602,-0.059271,-0.001926,-0.114290,1.000000,-0.015533,-0.155730,0.000632,0.993592,-0.015509,0.001894
long,-0.000676,-0.048278,-0.000187,-0.909732,-0.015533,1.000000,-0.052715,-0.000642,-0.015452,0.999120,0.001721
city_pop,-0.001678,-0.008991,0.005818,0.078467,-0.155730,-0.052715,1.000000,-0.001714,-0.154781,-0.052687,0.002136
unix_time,0.998971,0.000354,-0.000293,0.000670,0.000632,-0.000642,-0.001714,1.000000,0.000561,-0.000635,-0.005078
merch_lat,0.000541,-0.058942,-0.001873,-0.113561,0.993592,-0.015452,-0.154781,0.000561,1.000000,-0.015431,0.001741
merch_long,-0.000671,-0.048252,-0.000151,-0.908924,-0.015509,0.999120,-0.052687,-0.000635,-0.015431,1.000000,0.001721


In [6]:
sort = correlation.iloc[-1,:].sort_values(ascending = True)
sort

unix_time    -0.005078
Unnamed: 0   -0.004767
zip          -0.002162
cc_num       -0.000981
long          0.001721
merch_long    0.001721
merch_lat     0.001741
lat           0.001894
city_pop      0.002136
amt           0.219404
is_fraud      1.000000
Name: is_fraud, dtype: float64

In [7]:
drop = ['Unnamed: 0','trans_date_trans_time','first','last','zip','lat','long','city_pop','dob','trans_num','unix_time','merch_long','merch_lat','amt','is_fraud']
data = df.drop(columns = drop)
data


,cc_num,merchant,category,gender,street,city,state,job
0,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,F,561 Perry Cove,Moravian Falls,NC,"Psychologist, counselling"
1,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,F,43039 Riley Greens Suite 393,Orient,WA,Special educational needs teacher
2,38859492057661,fraud_Lind-Buckridge,entertainment,M,594 White Dale Suite 530,Malad City,ID,Nature conservation officer
3,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,M,9443 Cynthia Court Apt. 038,Boulder,MT,Patent attorney
4,375534208663984,fraud_Keeling-Crist,misc_pos,M,408 Bradley Rest,Doe Hill,VA,Dance movement psychotherapist
...,...,...,...,...,...,...,...,...
1296670,30263540414123,fraud_Reichel Inc,entertainment,M,162 Jessica Row Apt. 072,Hatch,UT,Geoscientist
1296671,6011149206456997,fraud_Abernathy and Sons,food_dining,M,8617 Holmes Terrace Suite 651,Tuscarora,MD,"Production assistant, television"
1296672,3514865930894695,fraud_Stiedemann Ltd,food_dining,M,1632 Cohen Drive Suite 639,High Rolls Mountain Park,NM,Naval architect
1296673,2720012583106919,"fraud_Reinger, Weissnat and Strosin",food_dining,M,42933 Ryan Underpass,Manderson,SD,Volunteer coordinator


In [8]:
x = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [10]:
test_data = pd.read_csv("fraudTest.csv")
test_data

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,555714,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,...,40.4931,-91.8912,519,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,1388534347,39.946837,-91.333331,0
555715,555715,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,...,29.0393,-95.4401,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0
555716,555716,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,...,46.1966,-118.9017,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0
555717,555717,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,...,44.6255,-116.4493,129,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,1388534364,44.470525,-117.080888,0


In [11]:
columns_drop = ['Unnamed: 0','trans_date_trans_time','first','last','zip','lat','long','city_pop','dob','trans_num','unix_time','merch_long','merch_lat','amt','is_fraud']
test = test_data.drop(columns=columns_drop)
test

,cc_num,merchant,category,gender,street,city,state,job
0,2291163933867244,fraud_Kirlin and Sons,personal_care,M,351 Darlene Green,Columbia,SC,Mechanical engineer
1,3573030041201292,fraud_Sporer-Keebler,personal_care,F,3638 Marsh Union,Altonah,UT,"Sales professional, IT"
2,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,F,9333 Valentine Point,Bellmore,NY,"Librarian, public"
3,3591919803438423,fraud_Haley Group,misc_pos,M,32941 Krystal Mill Apt. 552,Titusville,FL,Set designer
4,3526826139003047,fraud_Johnston-Casper,travel,M,5783 Evan Roads Apt. 465,Falmouth,MI,Furniture designer
...,...,...,...,...,...,...,...,...
555714,30560609640617,fraud_Reilly and Sons,health_fitness,M,558 Michael Estates,Luray,MO,Town planner
555715,3556613125071656,fraud_Hoppe-Parisian,kids_pets,M,572 Davis Mountains,Lake Jackson,TX,Futures trader
555716,6011724471098086,fraud_Rau-Robel,kids_pets,F,144 Evans Islands Apt. 683,Burbank,WA,Musician
555717,4079773899158,fraud_Breitenberg LLC,travel,M,7020 Doyle Stream Apt. 951,Mesa,ID,Cartographer


In [12]:
xtest = data.iloc[:,:-1]
ytest = data.iloc[:,-1]

In [14]:
prediction = Rand_forst_class.predict(xtest)
prediction
acc = accuracy_score(ytest,prediction)
print(f"Accuracy: {acc:}")